# How to use SLRealzer's own deblend method

In [1]:
import om10
import desc.slrealizer as sl
import pandas as pd
import warnings
warnings.filterwarnings("ignore")
from astropy.io import fits
import numpy as np
%matplotlib inline
%pylab inline
matplotlib.use('TkAgg')
matplotlib.rc('text', usetex=False)
%load_ext autoreload
%autoreload 2

Populating the interactive namespace from numpy and matplotlib


First, define filenames that will become our lens catalog and observation history.

In [2]:
catalog_f = '../data/qso_mock.fits'
observation_f = '../data/twinkles_observation_history.csv'

We make some initial cuts on the OM10 catalog to only work with LSST-like systems. The OM10 DB object will be an input to the SLRealizer constructor.

In [3]:
# Initialize om10
db = om10.DB(catalog=catalog_f)
# Select LSST-like lenses by querying with these values
# TODO make the values globally configurable
db.select_random(maglim=23.3, area=5000.0, IQ=0.75) # fwhm of median psf in arcsec
# Calculate the synthetic magnitude for the OM10 catalog -- need this
db.paint(synthetic=True)
# Calculate the sizes for the OM10 catalog -- if you want to assume the point-source-like-galaxies, get rid of this
#db.calculate_size()

OM10: Full db.lenses table contains 15658 systems
OM10: Initial db.sample contains 15658 systems
OM10: selection yields 781 lenses


The other input to the SLRealizer constructor is the observation history dataframe. We look at the first three years for an initial first pass. Here's what the observational history table looks like:

In [4]:
obs = pd.read_csv(observation_f)
obs = obs.query("(expMJD < 60919) & (filter != 'y')").reset_index(drop=True)
obs.head()

,obsHistID,expMJD,filter,FWHMeff,fiveSigmaDepth
0,183767,59823.286523,g,1.093153,24.377204
1,183811,59823.307264,g,1.231930,24.289872
2,184047,59823.418685,z,0.908511,21.923566
3,185595,59825.256044,r,0.949096,24.128617
4,185736,59825.325979,g,1.242407,24.316968


Now we can initialize SLRealizer with OM10 lens systems and Twinkles observations.

In [5]:
realizer = sl.SLRealizer(catalog=db, observation=obs)

Now we are ready to use SLRealizer to visualize the lens system. In the following cell, we take the first system (`lens_idx = 0`). First, we visualize the system by drawing each object of the system based on the catalog information and overlaying them on on a pixelated canvas -- this is the true model image. Then, after numerically obtaining the flux, position, size, and ellipticity of this image, we draw the system purely from the numerically derived parameters, yielding the emulated image. The pixel scale for both images is 0.2 arcsec/pixel.

In [14]:
#lens_idx = 0
#params = realizer.draw_lens_random_date(lensID=db.sample[lens_idx]['LENSID'])

The numerically derived parameters are stored in a dictionary called `params`. Let's take a look at its contents:

In [25]:
#params